In [4]:
!pip install langchain pypdf
!pip install --upgrade transformers bitsandbytes huggingface_hub accelerate
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader, WebBaseLoader, PyPDFLoader

# Define the BitsAndBytesConfig with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with the custom config
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create a text-generation pipeline using the quantized model
text_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 2048,
)

# Create a LangChain LLM using the HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=text_pipeline)

def summarize_text(text):
    prompt = ChatPromptTemplate.from_template(
        '''
        Summarize this text: {text}
        '''
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    summary = chain.run(text)
    return summary

def summarize_pdf(file):
    chain = load_summarize_chain(llm, chain_type="stuff")
    loader = PyPDFLoader(file)
    docs = loader.load()
    summary = chain.run(docs)
    return summary

def summarize_web(web):
    loader = WebBaseLoader(web)
    docs = loader.load()
    chain = load_summarize_chain(llm, chain_type="stuff")
    return chain.run(docs)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
summarize_text(
'''
Andrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur focusing on machine learning and artificial intelligence (AI).[2] Ng was a cofounder and head of Google Brain and was the former Chief Scientist at Baidu, building the company's Artificial Intelligence Group into a team of several thousand people.[3]

Ng is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL). Ng has also made substantial contributions to the field of online education as the cofounder of both Coursera and DeepLearning.AI.[4] He has spearheaded many efforts to "democratize deep learning" teaching over 2.5 million students through his online courses.[5][2] He is one of the world's most famous and influential computer scientists being named one of Time magazine's 100 Most Influential People in 2012, and Fast Company's Most Creative People in 2014. In 2018, he launched and currently heads the AI Fund, initially a $175-million investment fund for backing artificial intelligence startups. He has founded Landing AI, which provides AI-powered SaaS products.[6]

''')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Human: \n        Summarize this text: \nAndrew Yan-Tak Ng (Chinese: 吳恩達; born 1976) is a British-American computer scientist and technology entrepreneur focusing on machine learning and artificial intelligence (AI).[2] Ng was a cofounder and head of Google Brain and was the former Chief Scientist at Baidu, building the company\'s Artificial Intelligence Group into a team of several thousand people.[3]\n\nNg is an adjunct professor at Stanford University (formerly associate professor and Director of its Stanford AI Lab or SAIL). Ng has also made substantial contributions to the field of online education as the cofounder of both Coursera and DeepLearning.AI.[4] He has spearheaded many efforts to "democratize deep learning" teaching over 2.5 million students through his online courses.[5][2] He is one of the world\'s most famous and influential computer scientists being named one of Time magazine\'s 100 Most Influential People in 2012, and Fast Company\'s Most Creative People in 2014. In

In [9]:
# With link
summarize_web(
    'https://medium.com/@ravionaldoraffel/learn-english-pronunciation-with-the-help-of-llama-whisper-and-melotts-on-google-colab-85a364347f37'
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Write a concise summary of the following:\n\n\n"Learn English Pronunciation with the help of Llama, whisper, and MeloTTS on google colab. | by Raffel Ravionaldo | Sep, 2024 | MediumOpen in appSign upSign inWriteSign upSign inLearn English Pronunciation with the help of Llama, whisper, and MeloTTS on google colab.Raffel Ravionaldo·Follow7 min read·22 hours ago--ListenShareEnglish is an international language used in almost every region, and mastering this language is a must if we want to visit a country. However, sometimes we are not confident with our English pronunciation and want to learn how to pronounce It correctly. For that reason, I tried to create a Python code using several AI models so that we could learn pronunciation.For our program roadmap, it will look like this :Generative AI generates sentences that the user must read, on this article I use Llama 3B.The user records while reading the generated sentences, and sends them to the speech-to-text model. In this article, I us

In [10]:
summarize_pdf(
  '/content/Raffel Ravionaldo-resume.pdf'
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Write a concise summary of the following:\n\n\n"RAFFEL  RAVIONALDO  \nPhone : (+62)  85805759592  | Email : ravionaldoraffel@gmail.com  | LinkedIn  | GitHub  | Hugging  Face | Medium  \nI am a n Artificial Intelligence (AI) professional  with substantial experience as a computer vision engineer . My expertise includes using Python \nlibraries such as Scikit -learn and XGBoost for machine learning, implementing YOLOv8 for object detection and pose estimation, and developing \nimage and text classification models using TensorFlow and PyTorch. I have also explored advanced AI techniques like LangChain  for RAG, fine -\ntuning Stable Diffusion, and leveraging LLM  models  at hugging face . \nWORK EXPERIENCE  \nBroox Technologies  – Barcelona, Spain   Apr 2024 – Present  \nComputer Vision Engineer  (Remote)  \n• Conducted research and development for computer vision products, including crowd counting, age -gender detection, and shoplifting detection.  \n• Developed and optimized image clas